<a href="https://colab.research.google.com/github/Weedmastercrucifer/Neural-Networks-and-Fuzzy-Logic/blob/main/Assignment1Details/Assignment1_Q9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 1 Q9

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Assignment1NNFL/Copy of data_q6_q7.txt',delimiter="\t",error_bad_lines=False)
#removing bad lines from the dataset

insts = data.to_numpy()
m= len(insts[:,0]) #No of instances
n = len(insts[0,:])-1 #no of features  
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)

b'Skipping line 8: expected 8 fields, saw 10\nSkipping line 36: expected 8 fields, saw 10\nSkipping line 61: expected 8 fields, saw 9\nSkipping line 69: expected 8 fields, saw 9\nSkipping line 107: expected 8 fields, saw 9\nSkipping line 136: expected 8 fields, saw 9\nSkipping line 170: expected 8 fields, saw 9\nSkipping line 171: expected 8 fields, saw 9\nSkipping line 173: expected 8 fields, saw 9\nSkipping line 202: expected 8 fields, saw 9\nSkipping line 204: expected 8 fields, saw 9\n'


In [ ]:
def cal_prior(insts,c): #instances of training set , no of classes to be classified
  p= np.ones(c)
  l = len(insts[:,-1])
  for i in range(c):
    p[i] = np.count_nonzero(insts[:,-1] == i+1)/l
  return p # returns prior probabilities matrix



In [ ]:
def cal_mean(insts,m,n): 
  mean = np.ones(n)
  for i in range(n):
    mean[i] = np.sum(insts[:,i])/m
  return mean

In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  P_y = cal_prior(insts_tr,3) #prior probability vector
  
  insts_y1 = insts_tr[insts_tr[:,-1]==1] #instances for class 1 in training set
  insts_y2 = insts_tr[insts_tr[:,-1]==2] #instances for class 2 in training set
  insts_y3 = insts_tr[insts_tr[:,-1]==3] #instances for class 3 in training set
  mean1 = cal_mean(insts_y1,len(insts_y1[:,0]),n) #mean vector for class 1
  mean2 = cal_mean(insts_y2,len(insts_y2[:,0]),n) #mean vector for class 2
  mean3 = cal_mean(insts_y3,len(insts_y3[:,0]),n) #mean vector for class 3
  cov1  = np.cov((insts_y1[:,0:n].astype(np.float32)).T)
  cov2  = np.cov((insts_y2[:,0:n].astype(np.float32)).T)
  cov3  = np.cov((insts_y3[:,0:n].astype(np.float32)).T)
  coeff1 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov1)**0.5)
  coeff2 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov2)**0.5)
  coeff3 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov3)**0.5)
  y_pred = []
  y_acu = insts_te[:,-1]
  l = np.ones(3)
  for i in range(m_te):
    l[0] = coeff1*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean1),np.linalg.inv(cov1)),(insts_te[i,0:n] - mean1).T))
    l[1] = coeff2*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean2),np.linalg.inv(cov2)),(insts_te[i,0:n] - mean2).T))
    l[2] = coeff3*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean3),np.linalg.inv(cov3)),(insts_te[i,0:n] - mean3).T))
    MAP = np.ones(3) # maximum a posteriori
    e = l[0]*P_y[0] + l[1]*P_y[1] + l[2]*P_y[2] #evidence
    for i in range(3):
      MAP[i] = (l[i]*P_y[i])/e
    y_pred.append(1+ np.argmax(MAP))
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pred, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  overall_accuracy += (confmat[0][0] + confmat[1][1] + confmat[2][2])/sum(sum(confmat))
  accuracy_class1 += confmat[0][0]/sum(confmat[0])
  accuracy_class2 += confmat[1][1]/sum(confmat[1])
  accuracy_class3 += confmat[2][2]/sum(confmat[2])
print('Overall Accuracy:', overall_accuracy/K ,'\nClass1 Accuracy:',accuracy_class1/K,'\nClass2 Accuracy:',accuracy_class2/K,'\nClass3 Accuracy:',accuracy_class3/K ) 




Overall Accuracy: 0.9493589743589744 
Class1 Accuracy: 0.9077690936514466 
Class2 Accuracy: 0.9764705882352942 
Class3 Accuracy: 0.9651515151515151
